# Smart Room Controller (with a Door Sensor)

Instead just controlling the light (smart_light_controller_v1), here we extend the logic both the light and the fan using a simple door sensor.

## Scenario:
1. **Presence Sensor ($A$):** 1 if someone is detected, 0 otherwise,
2. **Ambient Light Sensor ($B$):** 1 if bright, 0 otherwise,
3. **Door Sensor ($C$):** 1 if the door is **open**,  and 0 if the door is **closed**.

## Desired Behavior:
- The light should turn on if someone is present and if it is dark,
- The **fan** should turn on only if the **door is closed** and **someone is present.**

## Circuit Design:
1. **Light ($L$):** Turn on when someone is present, and it is dark:
                $$L = A \cdot \overline{B}$$
2. **Fan ($F$):** Turns on when someone is present, and the the door is closed:
                $$F = A \cdot \overline{C}$$

Where:
* $\overline{B}$ is NOT of $B$,
* $\overline{C}$ is NOT of $C$.

## Real-World Meaning:
* The Light still follows our previous logic.
* The Fan should not run if the door is open (to save energy and prevent air loss).
* In real electronics:
    - The NOT Gate inverts sensor readings.
    - The AND Gate ensures correct conditions.
    - Transistors or Relays control real devices.

## ASCII Circuit Diagram:
```lua
       Presence Sensor (A)
                |
                v
             [ AND ]-----> Light (L)
                ^
                |
Ambient Light Sensor (B)
                |
             [ NOT ]  
                |
                v
       (Inverted Ambient Light)

=================================

       Presence Sensor (A)
                |
                v
             [ AND ]-----> Fan (F)
                ^
                |
       Door Sensor (C)
                |
             [ NOT ]
                |
                v
        (Inverted Door Signal)
```

In [1]:
def smart_room(presence: int,
               ambient_light: int,
               door: int
               ) -> tuple[int, int]:
    """
    Smart Room Controller:
        - presence: 1 if someone is detected, 0 otherwise,
        - ambient_light: 1 if it is bright, 0 otherwise,
        - door: 1 if it is open, 0 if closed

    return:
    - light = presence & not_ambient       
    - fan = presence & not_door
    """
    # NOT gates
    not_ambient: int = 1 - ambient_light
    not_door: int = 1 - door

    # AND gates
    light: int = presence & not_ambient
    fan: int = presence & not_door

    return light, fan

# Test different conditions:
TEST_CONDITIONS: list[tuple[int, int, int]] = [
    (0, 0, 0),  # No one present, dark, door closed
    (0, 1, 1),  # No one present, bright, door open
    (1, 0, 0),  # Someone present, dark, door closed
    (1, 1, 0),  # Someone present, bright, door closed
    (1, 0, 1),  # Someone present, dark, door open
    (1, 1, 1)   # Someone present, bright, door open
    ]
print("Presence, Ambient Light, Door => Light, Fan")
for presence, ambient, door in TEST_CONDITIONS:
    LIGHT, FAN = smart_room(presence, ambient, door)
    print(f"{presence}, {ambient}, {door} => {LIGHT}, {FAN}")


Presence, Ambient Light, Door => Light, Fan
0, 0, 0 => 0, 0
0, 1, 1 => 0, 0
1, 0, 0 => 1, 1
1, 1, 0 => 0, 1
1, 0, 1 => 1, 0
1, 1, 1 => 0, 0


# Smart Light with a Memory (Flip-Flop)
In the previous example, the system reacted instantly to inputs. But what if we want the light to stay on until someone leaves the room, even if the sensors change?

This requires **memory**, and the simplest **memory unit** is a **flip-flop**.

## Why Do We Need Memory?
In real situation, a light switch does not always work like AND gate. Sometimes, we press it **once to turn the light on** and **once to turn it off**, even it the room conditions change.

A **flip-flop** gives us this ability by **remembering the last state**.

## Choosing the Right Flip-Flop:
The simplest what to introduce memory is using **SR Latch (Set-Rest Flip-Flop)**:
- **Set ($S = 1$)** -> Turns ON,
- **Unset ($R = 1$)** -> Turns OFF,
- **$S = 0 \text{and} R = 0$** -> Keeps the last state.

## Scenario: Smart Light with Memory:
1. **Presence Sensor (A):** 1 if someone is detected, 0 otherwise,
2. **Light Switch (B):** 1 if pressed, 0 otherwise.

## Desired Behavior:
- If $A = 1$ (someone enters), the light **turns ON** and stays on,
- If $B = 1$ (Button pressed), the light **turns OFF**,
- If neither happens, the light **remembers its last state.**

## Circuit Design: SR Flip-Flop
The light's behavior can be modeled with an **SR Flip-flop**:
                $$Q_{text{next}} = S + \overline{R} \cdot Q$$
Where:
- **$S$ (Set) = $A$** (Presence Sensor sets the light ON),
- **$R$ (Reset) = $B$** (Button turns it OFF),
- **$Q$** is the current light state.

## ASCII Circuit Diagram:
```lua
        Presence Sensor (A)
                |
                v
             [ OR ]-----> S (Set)
                |
Light (Q) ----[ AND ]  
                ^
                |
      NOT(Button) (R)
                |
                v
     Button Press (B)
```

In [ ]:
class SmartLightSR:
    """
    Simulate the SR flip-flop of the light
    Q = 0 means off
    """

    def __init__(self) -> None:
        self.light: int = 0  # Q (initially off)

    def update(self,
               presence: int,
               button: int
               ) -> None:
        """
        Smart Light with Memory (SR Flip-Flop):
        - presence (A): 1 if someone enters, 0 otherwise (Set)
        - button (B): 1 if button is pressed, 0 otherwise (Reset)

        return: light state (1 = ON, 0 = OFF)
        """
        latch_s: int = presence
        latch_r: int = button

        # Flip-flop logic:
        if latch_s == 1:  # If someone there
            self.light = 1  # Turn light ON
        elif latch_r == 1:  # Reset: turn light OFF
            self.light = 0

smart_light = SmartLightSR()

# Conditions:
TEST_CONDITIONS = [
    (0, 0),  # No one, no button press (Light stays off)
    (1, 0),  # Someone enters (Light turns ON)
    (0, 0),  # Person leaves, but light stays ON (Memory effect)
    (0, 1),  # Button pressed (Light turns OFF)
    (0, 0),  # Light remains OFF until triggered again
    (1, 0),  # Someone enters again (Light turns ON)
]

print("Presence, Button => Light")
for presence, button in TEST_CONDITIONS:
    smart_light.update(presence, button)
    print(f"{presence}, {button} => {smart_light.light}")

Presence, Button => Light
0, 0 => 0
1, 0 => 1
0, 0 => 1
0, 1 => 0
0, 0 => 0
1, 0 => 1


### **How the Controller Reacts**
| Step | Presence (A) | Button (B) | Light (Q) |
|------|------------|----------|----------|
| **1**  | 0 (empty room) | 0 (button untouched) | 0 (light off) |
| **2**  | 1 (someone enters) | 0 (button untouched) | **1 (light on)** |
| **3**  | 0 (person leaves) | 0 (button untouched) | **1 (light stays on)** (because of memory) |
| **4**  | 0 (empty room) | 1 (button pressed) | **0 (light turns off)** |
| **5**  | 0 (empty room) | 0 (button untouched) | 0 (light remains off) |
| **6**  | 1 (someone enters) | 0 (button untouched) | **1 (light turns on again)** |

---
